<a href="https://colab.research.google.com/github/sharonwu827/Deep-Learning/blob/master/Semantic_Score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# load dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import package
!pip install transformers
!pip install bert-tensorflow 
!pip install pyenchant
!pip install -U pip setuptools wheel
!pip install -U spacy
!sudo apt-get install libenchant1c2a
!python -m spacy download en_core_web_sm

import warnings
warnings.filterwarnings('ignore')

import numpy as np 
from numpy import asarray
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer

import re
import time
import torch
import transformers as ppb

from nltk.corpus import stopwords
import torch
import torch.nn as nn
import transformers
from transformers import BertTokenizer, BertModel, AutoModel, BertTokenizerFast,BertForSequenceClassification

# Keras functional API
from keras.models import Sequential. Model, load_model, model_from_config
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, Flatten, Lambda
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K

import os
import csv
import copy

import tensorflow as tf
import tensorflow_hub as hub

import enchant #  Enchant spellchecking library
import spacy
import re

import urllib.request
from bs4 import BeautifulSoup
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip -q glove.6B.zip

     |████████████████████████████████| 2.5MB 13.6MB/s 
     |████████████████████████████████| 3.3MB 41.4MB/s 
     |████████████████████████████████| 901kB 38.8MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 71kB 5.6MB/s 
     |████████████████████████████████| 61kB 6.3MB/s 
     |████████████████████████████████| 1.6MB 15.7MB/s 
     |████████████████████████████████| 819kB 53.0MB/s 
Requirement already up-to-date: wheel in /usr/local/lib/python3.7/dist-packages (0.36.2)
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
  Found existing installation: setuptools 57.0.0
    Uninstalling setuptools-57.0.0:
      Successfully uninstalled setuptools-57.0.0


     |████████████████████████████████| 6.4 MB 12.2 MB/s 
     |████████████████████████████████| 456 kB 60.2 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 10.1 MB 54.4 MB/s 
     |████████████████████████████████| 619 kB 63.8 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The fol

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/asap-aes/training_set_rel3.tsv",sep='\t', encoding='ISO-8859-1')
dev = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/asap-aes/valid_set.tsv",sep='\t', encoding='ISO-8859-1')
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/asap-aes/test_set.tsv",sep='\t', encoding='ISO-8859-1')

In [ ]:
prompt = pd.DataFrame({"essay_set":[1,2,3,4,5,6,7,8],
                       "prompt":["More and more people use computers, but not everyone agrees that this benefits society. Those who support advances in technology believe that computers have a positive effect on people. They teach hand-eye coordination, give people the ability to learn about faraway places and people, and even allow people to talk online with other people. Others have different ideas. Some experts are concerned that people are spending too much time on their computers and less time exercising, enjoying nature, and interacting with family and friends. Write a letter to your local newspaper in which you state your opinion on the effects computers have on people. Persuade the readers to agree with you.",
                                "Write a persuasive essay to a newspaper reflecting your vies on censorship in libraries. Do you believe that certain materials, such as books, music, movies, magazines, etc., should be removed from the shelves if they are found offensive? Support your position with convincing arguments from your own experience, observations, and/or reading.",
                                "Write a response that explains how the features of the setting affect the cyclist. In your response, include examples from the essay that support your conclusion.",
                                "Write a response that explains why the author concludes the story with this paragraph. In your response, include details and examples from the story that support your ideas.",
                                "Describe the mood created by the author in the memoir. Support your answer with relevant and specific information from the memoir.",
                                "Based on the excerpt, describe the obstacles the builders of the Empire State Building faced in attempting to allow dirigibles to dock there. Support your answer with relevant and specific information from the excerpt.",
                                "Write about patience. Being patient means that you are understanding and tolerant. A patient person experience difficulties without complaining.Do only one of the following: write a story about a time when you were patient OR write a story about a time when someone you know was patient OR write a story in your own way about patience.",
                                "We all understand the benefits of laughter. For example, someone once said, “Laughter is the shortest distance between two people.” Many other people believe that laughter is an important part of any relationship. Tell a true story in which laughter was one element or part."]})

# Data processing

In [ ]:
def preprocess(df):
  df['normalized_score'] = df['domain1_score'] / df.groupby('essay_set')['domain1_score'].transform('max')

preprocess(train)

In [ ]:
def clean_anonymization(essay):
  '''
  function to remove the anoymaization
  '''
  res=[]
  # Lowercase
  essay = essay.lower()
  # Replace everything not a letter or apostrophe with a space
  essay = re.sub('[^a-zA-Z\']', ' ', essay)
  # Remove single letter words
  essay = ' '.join( [w for w in essay.split() if len(w)>1] )
  for i in essay.split():
    if i.startswith("@"):
      continue
    else:
      res.append(i)
  return ' '.join(res)

train['essay']=train['essay'].apply(lambda x:clean_anonymization(x))
# also remove from dev and test
dev['essay']=dev['essay'].apply(lambda x:clean_anonymization(x))
test['essay']=test['essay'].apply(lambda x:clean_anonymization(x))

In [ ]:
#-------------- change here------------------#
X = train[:100][['essay']]
y = train[:100]['normalized_score']

In [ ]:
def lstm_model():
    """
    Define the model
    """
    model = Sequential()

    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1,768], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model

In [ ]:
all_score = []
cv = KFold(n_splits=5, shuffle=True)
cv_data = cv.split(X)
results = []
prediction_list = []
fold_count =1
cuda = torch.device('cuda')

with torch.cuda.device(cuda):
  for traincv, testcv in cv_data:
    torch.cuda.empty_cache()
    print("\n--------Fold {}--------\n".format(fold_count))
    
    # get the train and test from the dataset.
    X_train, X_test, y_train, y_test = X.iloc[traincv], X.iloc[testcv], y.iloc[traincv], y.iloc[testcv]
    train_essays = X_train['essay']
    test_essays = X_test['essay']
    
    tokenized_train = train_essays.apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length=200)))
    tokenized_test = test_essays.apply((lambda x: tokenizer.encode(x, add_special_tokens=True ,max_length=200)))
    
    # train dataset
    max_len = 0
    for i in tokenized_train.values:
      if len(i) > max_len:
        max_len = len(i)
    padded_train = np.array([i + [0]*(max_len-len(i)) for i in tokenized_train.values])
    
    attention_mask_train = np.where(padded_train != 0, 1, 0)
    train_input_ids = torch.tensor(padded_train)
    train_attention_mask = torch.tensor(attention_mask_train)
    
    with torch.no_grad():
      last_hidden_states_train = model(train_input_ids, attention_mask=train_attention_mask, )
    train_features = last_hidden_states_train[0][:,0,:].numpy()
    
    ## test dataset
    max_len = 0
    for i in tokenized_test.values:
      if len(i) > max_len:
        max_len = len(i)
    padded_test = np.array([i + [0]*(max_len-len(i)) for i in tokenized_test.values])
    
    attention_mask_test = np.where(padded_test != 0, 1, 0)
    test_input_ids = torch.tensor(padded_test)  
    test_attention_mask = torch.tensor(attention_mask_test)

    with torch.no_grad():
      last_hidden_states_test = model(test_input_ids, attention_mask=test_attention_mask)
    test_features = last_hidden_states_test[0][:,0,:].numpy()
    train_x,train_y = train_features.shape
    test_x,test_y = test_features.shape

    x_train_reshaped = np.reshape(train_features,(train_x,1,train_y))
    x_test_reshaped = np.reshape(test_features,(test_x,1,test_y))

    lstm = lstm_model()
    lstm.fit(x_train_reshaped, y_train, batch_size=128, epochs=70)
    
    y_pred = lstm.predict(x_test_reshaped)
    
    # evaluate the model
    result = mean_squared_error(y_test.values,y_pred)
    print("MSE: {}".format(result))
    results.append(result)
    fold_count +=1

    tf.keras.backend.clear_session()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
all_score.append(results)
print("mean squared error is : {}".format(np.mean(np.asarray(results))))